In [ ]:
import folium
import keplergl
import json
import geopandas as gpd
import pandas as pd
import osmnx as ox
from glob import glob

from tqdm.notebook import trange, tqdm

Read file containing the cities polygons

In [ ]:
cities_gdf = gpd.read_file("cities/Case_studies_limits_final.shp")

Extract the selected entries corresponding to the selected cities

In [ ]:
cities = list(cities_gdf['Name'])

In [ ]:
cities

Iterate through the cities and sample points

In [ ]:
for city_name in cities:
    print('City: ', city_name, '__')
    # ox.graph_from_place does not work for Copenhagen. Use different function
    if city_name == 'Copenhagen':
        continue
        G = ox.graph_from_polygon(cities_gdf.iloc[16]['geometry'],
                        network_type='drive')
    
    else:
        if city_name != 'London':
            continue
        
        G = ox.graph_from_place(city_name,
                                network_type='drive')
    
    G_proj = ox.project_graph(G)
    total_drive_length = ox.basic_stats(G_proj, clean_intersects=True, circuity_dist='euclidean')['edge_length_total']
    print('\tTotal length of the road network:', total_drive_length)
    
    n_points = int((total_drive_length/1000) * 10)
    print('\tNumber of points to be estimated:', n_points)
    break
    points = ox.utils_geo.sample_points(ox.get_undirected(G), n_points*2)
    
    data = pd.DataFrame()
    data["Longitude_A"] = points[:int(n_points)].reset_index(drop=True).x
    data["Latitude_A"]  = points[:int(n_points)].reset_index(drop=True).y
    data["Longitude_B"] = points[int(n_points):].reset_index(drop=True).x
    data["Latitude_B"]  = points[int(n_points):].reset_index(drop=True).y
    print('\tData shape: ', data.shape)
    
    data.to_csv('city_points-on_streets/points_{}.csv'.format(city_name))

Confirm output files

In [ ]:
for csv_file in sorted(glob('city_points-on_streets/*.csv')):
       print('File:', csv_file.ljust(50), 'City:', csv_file.split('_')[-1].replace('.csv', '').ljust(20),'Points:', pd.read_csv(csv_file).shape[0])

Read one csv output and plot the points

In [ ]:
data = pd.read_csv('city_points-on_streets/points_{}.csv'.format('Copenhagen'))
print('Total number of points sampled:', data.shape[0])

In [ ]:
points_to_draw = 1500
m = folium.Map(location=[data['Latitude_A'].mean(), data['Longitude_A'].mean()], zoom_start=7, tiles='CartoDB positron')

for _, row in data[:points_to_draw].iterrows():
    folium.CircleMarker([row['Latitude_A'], row['Longitude_A']],
                       radius=2).add_to(m)
m